# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
conda install -c conda-forge gmaps

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
output_data_file = "../starter_code/weather_df.csv"
# output_data_file

types_df = pd.read_csv(output_data_file)

weather_data_df = types_df.drop(columns=["Unnamed: 0"])
weather_data_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,atuona,0,PF,1602110313,75,-9.80,-139.03,77.92,15.55
1,taoudenni,39,ML,1602110313,14,22.68,-3.98,92.61,17.36
2,tocopilla,20,CL,1602110313,62,-22.09,-70.20,64.24,10.49
3,tiksi,0,RU,1602110313,94,71.69,128.87,17.42,12.66
4,busselton,7,AU,1602110313,88,-33.65,115.33,50.00,5.95
...,...,...,...,...,...,...,...,...,...
549,sioux lookout,1,US,1602110376,27,42.73,-96.23,79.00,10.29
550,ujszasz,90,HU,1602110376,100,47.30,20.08,55.40,9.17
551,chirongui,90,YT,1602110376,83,-12.93,45.15,78.80,5.82
552,west plains,5,US,1602110176,33,31.80,-97.09,87.80,6.93


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [5]:
locations = weather_data_df[["Lat", "Lng"]]
# locations

humidity = weather_data_df["Humidity"].astype(float)

# humidity

# figure_layout = {
#     'width' : '1000px',
#     'height':'400px',
#     'border':'1px solid black',
#     'padding':'1px',
#     'margin':'0 auto 0 auto'
# }

# fig = gmaps.figure(layout=figure_layout)

fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                   dissipating=False, max_intensity=5,
                                   point_radius=1)

fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
new_types_df = pd.DataFrame(weather_data_df, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])

max_temp = (new_types_df["Max Temp"] <= 80) & (new_types_df["Max Temp"] > 70)
wind_speed = new_types_df["Wind Speed"] < 10
cloudiness = new_types_df["Cloudiness"] == 0

# new_types_df[max_temp & wind_speed & cloudiness]
new_types_df[max_temp & wind_speed & cloudiness]

,City,Max Temp,Wind Speed,Cloudiness
81,havelock,75.00,7.20,0
82,gerash,75.20,2.15,0
86,bahia honda,73.45,5.08,0
110,saint-philippe,71.60,8.05,0
126,maceio,73.40,8.05,0
146,buraydah,78.80,5.82,0
152,sur,77.77,7.02,0
228,saint-leu,71.60,8.05,0
245,rafaela,73.00,4.00,0
250,touros,77.00,5.82,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = pd.DataFrame(weather_data_df, columns=["City", "Country", "Lat", "Lng"])

hotel_df["Hotel Name"] = ""
# hotel_df["Coordinates"]=""
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,atuona,PF,-9.80,-139.03,
1,taoudenni,ML,22.68,-3.98,
2,tocopilla,CL,-22.09,-70.20,
3,tiksi,RU,71.69,128.87,
4,busselton,AU,-33.65,115.33,
...,...,...,...,...,...
549,sioux lookout,US,42.73,-96.23,
550,ujszasz,HU,47.30,20.08,
551,chirongui,YT,-12.93,45.15,
552,west plains,US,31.80,-97.09,


In [8]:
target_coordinates = f"{hotel_df['Lat'][0]},{hotel_df['Lng'][0]}"
target_search = "hotel"
target_radius = 5000
target_type = "hotel"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

In [9]:
pprint(response.json(), depth=1)

{'html_attributions': [], 'results': [...], 'status': 'OK'}


In [10]:
hotel_df.iloc[0][0]

'atuona'

In [16]:
#creates a list to capture hotel_names to insert into dataframe
hotel_name = []

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# taken from Airpost_Ratings Example done in class
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))
    try:
#         hotel_df.iloc[index, 4] = name_address['results'][0]['name']
        hotel_name.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)

In [17]:
hotel_name

['Hôtel Hiva Oa Hanakee Lodge',
 nan,
 'Hotel Royal Tocopilla',
 'Arktika',
 'Abbey Beach Resort',
 'Al shukri hotel',
 'Hotel Intercity Suape Costa Dourada',
 nan,
 'Bonthe Holiday Village',
 nan,
 "Smitty's Bed and Breakfast",
 'Pacific Resort Rarotonga',
 nan,
 'Arakur Ushuaia Resort & Spa',
 'Pestana',
 'Chautauqua Harbor Hotel',
 nan,
 nan,
 'Last Light Lodge',
 'Pebbles Inn',
 nan,
 'Hotel Krishna Residency',
 'K-POP HOTEL Seoul Tower',
 nan,
 'Miri Marriott Resort & Spa',
 nan,
 nan,
 'Era Spa',
 'Hotel Miramar',
 'Nuli Sapi',
 'Hotel Hacienda Vieja',
 'Talnakh Hotel',
 'InterContinental Bora Bora Resort Thalasso Spa',
 'Namatanai Hotel',
 'Hostal Molino Viejo',
 'Приватний гуртожиток',
 'One Marine Drive Boutique Hotel',
 'Hotel Castelo',
 'Hotel Kap Farvel',
 'Lukoil',
 'Gostinitsa "Ishim"',
 'Clove Island Zanzibar Villas & Spa',
 nan,
 'Osmani Hotel',
 'Eysturland Lodge',
 'Hotel Paraiso Bom Jesus Breves',
 nan,
 'Days Hotel & Suites Ivy Zunyi',
 "Ellie's Villa",
 'THE REDWOO

In [18]:
hotel_df['Hotel Name']= hotel_name
hotel_df.dropna()
hotel_df.to_csv('hotel_export.csv')

In [19]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [20]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))